<a href="https://colab.research.google.com/github/macintoxic/BootCampRendaVariavel/blob/main/trabalho_pratico_2/trabalho_pratico_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instala as dependencias

In [ ]:
!pip install apache-airflow
!pip install pandas_datareader --upgrade
!pip install sidrapy


## Imports

In [35]:

import yfinance as yf
import requests
import pandas as pd
import sidrapy as sidra
import sqlite3
from pandas_datareader import data
from sqlalchemy import create_engine 
import pendulum



## Colete a série histórica dos dados de inflação de bebidas do IBGE usando a API sidra

In [36]:
def get_ipca():
    url_ipca = 'https://apisidra.ibge.gov.br/values/t/1705/n1/all/v/355/p/all/c315/7169,7170,7389,7396,7397,7440,7443,12394/d/v355%202'

    ipca_raw = sidra.get_table(table_code=1705,
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            categories=[7169,7170,7389,7396,7397,7440,7443,12394]                           
                            )
    return ipca_raw


get_ipca().head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Mês (Código),Mês,Variável (Código),Variável,"Geral, grupo, subgrupo, item e subitem (Código)","Geral, grupo, subgrupo, item e subitem"
1,1,Brasil,2,%,0.53,1,Brasil,201202,fevereiro 2012,355,IPCA15 - Variação mensal,7169,Índice geral
2,1,Brasil,2,%,1.18,1,Brasil,201202,fevereiro 2012,356,IPCA15 - Variação acumulada no ano,7169,Índice geral
3,1,Brasil,,,...,1,Brasil,201202,fevereiro 2012,1120,IPCA15 - Variação acumulada em 12 meses,7169,Índice geral
4,1,Brasil,2,%,100.0000,1,Brasil,201202,fevereiro 2012,357,IPCA15 - Peso mensal,7169,Índice geral


## Colete a série histórica dos dados diários de preços e volume
(OHLCV) para a ação ABEV3 do provedor de sua preferência (yahoo
finance, marketstack, eodhistoricaldata, finnhub, alphavantage
etc.);


In [51]:
def get_abev3():
    yf.pdr_override()
    abv3_data  = data.get_data_yahoo("ABEV3.SA") 
    abv3_data['ticker'] = 'ABEV3.SA'

    return abv3_data

get_abev3().head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2000-01-05,0.520882,0.520882,0.520882,0.520882,0.305528,985,ABEV3.SA
2000-01-06,0.494478,0.494478,0.494478,0.494478,0.290040,227,ABEV3.SA
2000-01-07,0.494478,0.494478,0.494478,0.494478,0.290040,151,ABEV3.SA
2000-01-10,0.494478,0.494478,0.494478,0.494478,0.290040,1516,ABEV3.SA
2000-01-11,0.494478,0.494478,0.494478,0.494478,0.290040,3791,ABEV3.SA


## Crie e armazene esses dados em um banco de dados SQL local ou em nuvem

In [46]:
def save_to_sqlite():
    engine = create_engine('sqlite:////content/market_data.db', echo=False)
    get_abev3().to_sql('data', con=engine)


## Esboce como seria a implementação em Python de uma DAG Airflow, que  executaria as coletas implementadas anteriormente. O código não precisa rodar com sucesso, é apenas um esboço.

### Inicializa o airflow

In [42]:
!airflow db init

DB: sqlite:////root/airflow/airflow.db
[2023-04-06 01:45:40,966] {migration.py:207} INFO - Context impl SQLiteImpl.
[2023-04-06 01:45:40,966] {migration.py:210} INFO - Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 290244fb8b83
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done


### Imports do airflow

In [43]:
from airflow import DAG
from datetime import datetime, timedelta
from airflow.operators.python import PythonOperator

In [44]:
def get_ipca():
    url_ipca = 'https://apisidra.ibge.gov.br/values/t/1705/n1/all/v/355/p/all/c315/7169,7170,7389,7396,7397,7440,7443,12394/d/v355%202'

    ipca_raw = sidra.get_table(table_code=1705,
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            categories=[7169,7170,7389,7396,7397,7440,7443,12394]                           
                            )


    #ipca_raw.head()
    return ipca_raw



## Parâmetros default do DAG

In [45]:
args = {
            'owner': 'airflow',    
            #'start_date': airflow.utils.dates.days_ago(2),
            # 'end_date': datetime(),
            # 'depends_on_past': False,
            #'email': ['airflow@example.com'],
            #'email_on_failure': False,
            #'email_on_retry': False,
            # If a task fails, retry it once after waiting
            # at least 5 minutes
            #'retries': 1,
            'retry_delay': timedelta(minutes=5),
        }

## Instanciar o DAG

In [85]:

dag_python = DAG(
	dag_id = "pythonoperator_demo",
	default_args=args,
	# schedule_interval='0 0 * * *',
	schedule='@once',	
	dagrun_timeout=timedelta(seconds=1),
	description='Esboço de airflow',
	start_date =  pendulum.today('UTC').add(days=-1))

## Chamadas do DAG

In [86]:
python_task_ipca = PythonOperator(task_id='get_ipca', python_callable=get_ipca, dag=dag_python)
python_task_get_abev3 = PythonOperator(task_id='get_abev3', python_callable=get_abev3, dag=dag_python)
python_task_save_to_sqlite = PythonOperator(task_id='save_to_sqlite', python_callable=save_to_sqlite, dag=dag_python)
python_task_ipca >> python_task_get_abev3 >> python_task_save_to_sqlite



usage: airflow [-h] GROUP_OR_COMMAND ...

positional arguments:
  GROUP_OR_COMMAND

    Groups:
      dags          Manage DAGs
      tasks         Manage tasks

    Commands:

optional arguments:
  -h, --help        show this help message and exit
[2023-04-06 02:12:30,519] {ultratb.py:152} ERROR - Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/usr/lib/python3.9/argparse.py", line 1870, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib/python3.9/argparse.py", line 2082, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/usr/lib/python3.9/argparse.py", line 2038, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.9/argparse.py", line 1931, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib/python3.9/argparse.py", line 2472, in _get_values
    self.


airflow command error: argument GROUP_OR_COMMAND: invalid choice: '/root/.local/share/jupyter/runtime/kernel-a1a454c4-2645-4361-ab1c-8686c6ad4669.json' (choose from 'dags', 'tasks'), see help above.


TypeError: ignored

In [58]:
!pip install ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00


In [87]:
!airflow -h



usage: airflow
       [-h]
       GROUP_OR_COMMAND
       ...

positional arguments:
  GROUP_OR_COMMAND

    Groups:
      celery
    Celery
    components
      config
    View config
    uration
      connections
    Manage
    connections
      dags
    Manage DAGs
      db
    Database
    operations
      jobs
    Manage jobs
      kubernetes
    Tools to
    help run
    the Kuberne
    tesExecutor
      pools
    Manage
    pools
      providers
    Display
    providers
      roles
    Manage
    roles
      tasks
    Manage
    tasks
      users
    Manage
    users
      variables
    Manage
    variables

    Commands:
      cheat-sheet
    Display
    cheat sheet
      dag-processor
    Start a
    standalone
    Dag
    Processor
    instance
      info
    Show
    information
    about
    current
    Airflow and
    environment
      kerberos
    Start a
    kerberos
    ticket
    renewer
      plugins
    Dump
    information
    about
    loaded
    plugins
      rot

In [70]:
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
# <NgrokTunnel: "http://<public_sub>.ngrok.io" -> "http://localhost:80">
http_tunnel = ngrok.connect()
# Open a SSH tunnel
# <NgrokTunnel: "tcp://0.tcp.ngrok.io:12345" -> "localhost:22">
ssh_tunnel = ngrok.connect(22, "tcp")

usage: airflow [-h] GROUP_OR_COMMAND ...

positional arguments:
  GROUP_OR_COMMAND

    Groups:
      dags          Manage DAGs
      tasks         Manage tasks

    Commands:

optional arguments:
  -h, --help        show this help message and exit
[2023-04-06 02:08:25,686] {ultratb.py:152} ERROR - Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/usr/lib/python3.9/argparse.py", line 1870, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib/python3.9/argparse.py", line 2082, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/usr/lib/python3.9/argparse.py", line 2038, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.9/argparse.py", line 1931, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib/python3.9/argparse.py", line 2472, in _get_values
    self.


airflow command error: argument GROUP_OR_COMMAND: invalid choice: '/root/.local/share/jupyter/runtime/kernel-a1a454c4-2645-4361-ab1c-8686c6ad4669.json' (choose from 'dags', 'tasks'), see help above.


TypeError: ignored